In [78]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [79]:
#client_id - идентификатор клиента
#app_date - дата подачи заявки - (новый признак)
#education - уровень образования
#sex - пол заемщика
#age - возраст заемщика
#car - флаг наличия автомобиля
#car_type - флаг автомобиля иномарки
#decline_app_cnt - количество отказанных прошлых заявок
#good_work - флаг наличия “хорошей” работы
#score_bki - скоринговый балл по данным из БКИ- (новый признак)
#bki_request_cnt - количество запросов в БКИ
#region_rating - рейтинг региона- (новый признак)
#home_address - категоризатор домашнего адреса
#work_address - категоризатор рабочего адреса
#income - доход заемщика
#sna - связь заемщика с клиентами банка- (новый признак)
#first_time - давность наличия информации о заемщике- (новый признак)
#foreign_passport - наличие загранпаспорта
#default - флаг дефолта по кредиту

In [80]:
DATA_DIR = '/kaggle/input/sf-scoring/'
df_train = pd.read_csv(DATA_DIR +'/train.csv')
df_test = pd.read_csv(DATA_DIR +'/test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [81]:
sample_submission.shape

In [82]:
df_test.shape

In [83]:
df_train.info()

In [84]:
df_train.head(5)

In [85]:
df_test.info()

In [86]:
sample_submission.head(5)

In [87]:
sample_submission.info()

In [88]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0  # помечаем где у нас тест
df_test['default'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [89]:
data.nunique(dropna=False)

In [90]:
data['default'].value_counts(ascending=True).plot(kind='barh')

Выборка является несбаланстрованной, в качестве целевых метрик будем использовать F1-score 

In [91]:
num_cols = ['age', 'score_bki', 'decline_app_cnt', 'bki_request_cnt', 'income']
cat_cols = ['education', 'first_time', 'sna', 'work_address', 'home_address', 'region_rating']
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']

In [92]:
#Избавиться от лишних признаков
data = data.drop(['client_id','app_date'], axis = 1)

#Новые признаки на основе имеющихся сгенерировать сложно, так как значения большинства признаков являются условными

In [93]:
#Бинарные признаки преобразуем в 0 и 1 методом LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for column in bin_cols:
    data[column] = le.fit_transform(data[column])

In [94]:
#Смотрим числовые признаки
import matplotlib.pyplot as plt
for column in num_cols:
    data[column].hist()
    plt.xlabel(column);
    plt.show()

In [95]:
#Нормализуем числовые признаки
for column in num_cols:
    data[column] = (data[column] - data[column].min())/(data[column].max()- data[column].min())
    data[column].hist()
    plt.xlabel(column);
    plt.show()

In [96]:
#Оценим корреляцию Пирсона для непрерывных переменных
import seaborn as sns 
sns.heatmap(data[num_cols].corr().abs(),vmin=0, vmax=1)
#Как мы видим, взаимосвязь пар числовых признаков по Пирсону слабая. 
#Это очень здорово для нашей линейной модели!

In [97]:
#Для оценки значимости числовых переменных в качестве меры значимости мы будем использовать значение f-статистики.
from sklearn.feature_selection import f_classif

imp_num = pd.Series(f_classif(data[num_cols], data['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')
# признак skore_bki самый значимый из числовых

In [98]:
#категориальные признаки
data['education'].value_counts()

In [99]:
#Заполним пропуски в education строкой с самым популярным значением
data['education'] = data['education'].fillna(data['education'].value_counts().index[0])
data.info()

In [100]:
#Преобразуем education в численный формат
education_range = {'SCH': 1,
                   'GRD': 2,
                   'UGR': 3,
                   'PGR': 4,
                   'ACD': 5}
data['education'] = data['education'].apply(lambda x: education_range[x])

#Оценка значимости переменных
from sklearn.feature_selection import mutual_info_classif

imp_cat = pd.Series(mutual_info_classif(data[bin_cols + cat_cols], data['default'],
                                     discrete_features =True), index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [101]:
#Была сделана попытка улучшить качество модели за счет уменьшения количества признаков - удаления менее значимых,
#но в итоге значения метрик не изменились

num_cols = ['score_bki', 'decline_app_cnt', 'bki_request_cnt', 'income']
cat_cols = ['sna', 'first_time', 'region_rating', 'home_address', 'education', 'work_address']
bin_cols = ['car_type', 'good_work', 'foreign_passport']
data = data.drop(['age', 'sex', 'car'], axis=1)
data.info()

In [102]:
#Воспользуемся dummy-кодированием для категориальных переменных 
for column in cat_cols:
    data = pd.get_dummies(data, columns=[column], dummy_na=False)

In [103]:
data.info()

In [104]:
# Стандартизация числовых признаков методом StandardScaler
#from sklearn.preprocessing import StandardScaler
#X_num = StandardScaler().fit_transform(data[num_cols].values)
#data.loc[:,num_cols] = X_num

In [105]:
# Стандартизация числовых признаков 
for column in num_cols:
    data[column] = (data[column] - data[column].mean())/data[column].std()
    data[column].hist()
    plt.xlabel(column);
    plt.show()

In [106]:
data

In [107]:
# Импортируем необходимые функции:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # инструмент для создания и обучения модели
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve # инструменты для оценки точности модели

In [108]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample', 'default'], axis=1)

y = train_data['default'].values  # наш таргет
X = train_data.drop(['default'], axis=1)

In [109]:
test_data.info()

In [110]:
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [111]:
# проверяем
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [112]:
logreg = LogisticRegression(solver='liblinear', C = 0.01, max_iter=1000)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [113]:
#Оцениваем качество модели
#Так как выборка является несбаланстрованной, в качестве целевых метрик смотрим на F1-score 
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [114]:
# Посмотрим ROC кривую
y_pred_proba = logreg.predict_proba(X_test)[:, 1]

roc_auc = roc_auc_score(y_test, y_pred_proba)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

In [115]:
#Подбор гиперпараметров модели
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import GridSearchCV 

iter_ = 50
epsilon_stop = 1e-3

param_grid = [
    {'penalty': ['l1'], 
     'solver': ['liblinear', 'lbfgs'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['l2'], 
     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['none'], 
     'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
]

gridsearch = GridSearchCV(logreg, param_grid, scoring='f1', n_jobs=-1, cv=5)
gridsearch.fit(X_train, y_train)
logreg = gridsearch.best_estimator_

#печатаем параметры
best_parameters = logreg.get_params()
for param_name in sorted(best_parameters.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))

In [116]:
#Функция создания модели на заданном наборе параметров
def LogReg (parameters):
    model = LogisticRegression(C = parameters['C'],
                            class_weight = parameters['class_weight'],
                            dual = parameters['dual'],
                            fit_intercept = parameters['fit_intercept'],
                            intercept_scaling = parameters['intercept_scaling'],
                            l1_ratio = parameters['l1_ratio'],
                            max_iter = parameters['max_iter'],
                            multi_class = parameters['multi_class'],
                            n_jobs = parameters['n_jobs'],
                            penalty = parameters['penalty'],  
                            random_state = parameters['random_state'],
                            solver = parameters['solver'], 
                            tol = parameters['tol'],
                            verbose = parameters['verbose'],
                            warm_start = parameters['warm_start'])
    return model

In [117]:
#Создаем модель с оптимальными параметрами и обучаем
logreg = LogReg(best_parameters)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [118]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
#Метрики precision, recall, F1-score для 1 улучшились

In [119]:
#Сбалансируем обучающую выборку, уменьшив количество наблюдений со значением целевой переменной 0
n = train_data['default'].value_counts().min()
defaults = train_data['default'].value_counts(ascending = True).index          #нулевым элементом будет значение для класса-меньшинства, то есть для "1"
slice = train_data[train_data['default'] == defaults[1]][:n]                   #для класса-большинства, то есть для "0", берем количество наблюдений равное количеству наблюдений класса-меньшинства
slice_balance = slice.append(train_data[train_data['default'] == defaults[0]]) #объединяем наблюдения для двух классов
slice_balance['default'].value_counts(ascending=True).plot(kind='barh')

In [120]:
#Обучим созданную с оптимальными параметрами модель на новой выборке
y_balance = slice_balance['default'].values  
X_balance = slice_balance.drop(['default'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_balance, y_balance, test_size=0.2, random_state=42)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [121]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [122]:
#качество нас устраивает,
#либо создаем финальную модель и обучаем её на всей сбалансированной выборке
#logreg_final = LogReg(best_parameters)
#logreg_final.fit(X_balance, y_balance)

In [123]:
#либо обучаем нашу старую созданную с оптимальными параметрами модель на всех обучающих данных
logreg.fit(X, y)

In [124]:
#predict_submission = logreg_final.predict(test_data)
predict_submission = logreg.predict(test_data)

In [125]:
sample_submission['default'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

In [126]:
sample_submission.describe()

In [127]:
sample_submission['default'].value_counts(ascending=True).plot(kind='barh')

In [128]:
#!kaggle competitions submit -c sf-scoring -f ssubmission.csv -m "Project5.The computer says NO"
# !kaggle competitions submit your-competition-name -f submission.csv -m 'My submission message'